# stitch chromosomal contigs based on nucmer results
Input: a pad file & a fasta file
Sherwood's check (April 7, 2023):
Remove (somewhat messy) wraparound 4743 bp at left end.  Has telomere after this removal.
Has a right end extension somewhat similar to that of B31 but has no right end telomere consensus.  Currently no N’s at right end.

scw9.pad:

unitig_7|quiver|quiver|pilon    -1      1397
unitig_34|quiver|quiver|pilon   1       10
unitig_6|quiver|quiver|pilon    1       743
unitig_1|quiver|quiver|pilon    -1      1703
unitig_35|quiver|quiver|pilon   -1      1660
unitig_31|quiver|quiver|pilon   -1      0
unitig_36|quiver|quiver|pilon   1       556
unitig_5|quiver|quiver|pilon    1       277
unitig_42|quiver|quiver|pilon   1       -36
unitig_23|quiver|quiver|pilon   -1      2123
unitig_27|quiver|quiver|pilon   -1      1428
unitig_13|quiver|quiver|pilon   1       403
unitig_41|quiver|quiver|pilon   1       960
unitig_8|quiver|quiver|pilon    1       0

In [4]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

file_in ='Broken-to-be-gapped/SCW_9.chrom.fasta'
file_out='Broken-to-be-gapped/SCW-9-chrom-gapped-v2.fasta'

df_pad = pd.read_csv("Broken-to-be-gapped/scw9.pad", delimiter="\t", header=None)
df_pad.columns = ['seq_id', 'revcom', 'pad']
print(df_pad.head())
print(df_pad['pad'].sum())


                          seq_id  revcom   pad
0   unitig_7|quiver|quiver|pilon      -1  1397
1  unitig_34|quiver|quiver|pilon       1    10
2   unitig_6|quiver|quiver|pilon       1   743
3   unitig_1|quiver|quiver|pilon      -1  1703
4  unitig_35|quiver|quiver|pilon      -1  1660
11224


In [6]:
seqs = {}
for seq_record in SeqIO.parse(open (file_in, mode='r'), 'fasta'):
    seqs[seq_record.id] = seq_record

seq_str = ""
contig_len = 0
pad_len = 0
for index, row in df_pad.iterrows():
    id = row['seq_id']
    revcom = row['revcom']
    pad = row['pad']
    
    seq_contig = ''
    if revcom < 0:
        seq_contig = str(seqs[id].reverse_complement().seq)
    else:
        seq_contig = str(seqs[id].seq)
        
    contig_len += len(seqs[id].seq)
    
    # pad or merge
    if pad >= 0: # gap, pad N's
        for i in range(pad):
            seq_contig += 'N'
    else: # overlap, merge
        if revcom < 0:
            seq_contig = seq_contig[abs(pad):] # 4:end (inclusive from 5th, remove 4 bases from revcom start)
        else:
            seq_contig = seq_contig[:pad] # 0:-4 (remove 4 bases from seq end)          
    
    seq_str += seq_contig
    pad_len += pad

# remove wrap-around left 4743 bp:
seq_str = seq_str[4743:]

seq_out = SeqRecord(id = "SCW-9_chromosome_gapped", seq = Seq(seq_str))
# check sum:
print(contig_len)
print(pad_len)
print(len(seq_str))

with open(file_out, "w") as f_out:
    f_out.write(seq_out.format('fasta'))


903241
11224
909722
